# TDDA Introduction

## Test-Driven Data Analysis in Python

Eberhard Hansis @ PyData Meetup Munich, 2019

Many ideas presented here are borrowed from **Nicholas J. Radcliffe** of [Stochastic Solutions](http://stochasticsolutions.com/), and his colleagues. 

A longer introduction to the topic can be found at http://stochasticsolutions.com/pdf/tdda-tutorial-datafest-2019.pdf

Thanks to Nick for providing me with input for the slides!

## http://www.tdda.info

Note to self: compile notebook to slides with `jupyter nbconvert tdda-intro-2019.ipynb --to slides --reveal-prefix=reveal.js`

This example uses data from [Wikidata](https://www.wikidata.org).
Wikidata contains structured data on a huge variety of topics, which are linked in a knowledge graph.
Data is [public domain](https://creativecommons.org/publicdomain/zero/1.0), i.e. free for any type of use.

Data can be queried via the [Wikidata Query Service](https://query.wikidata.org/) using
the [SPARQL](https://en.wikipedia.org/wiki/SPARQL) query language.
The Query Service page contains links to helpful
[SPARQL help](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/Wikidata_Query_Help)
and examples.

In this example we will use some data about paintings, in particular portrait paintings.
The following query can be run in the [Wikidata Query Service](https://query.wikidata.org/).
It searches items whose genre (P136) is any subclass (P279*) of 'portrait' (Q134307). For these, it queries the creator (P170) as Wikidata link, width (P2049) and height (P2048)
in normalized units, and the date of inception (P571).
Since there may be many dates of inception (e.g. an earliest and latest estimated date),
a grouped query is executed and the minimum date selected. For the other fields,
a random sample is selected (from the, usually, single available value).
When writing this notbook, this returned data for 27000 paintings.

```
    SELECT ?painting ?paintingLabel
    (SAMPLE(?creator) AS ?creator)
    (SAMPLE(?width) AS ?width)
    (SAMPLE(?height) AS ?height)
    (MIN(?inception) AS ?inception)
    WHERE {
      ?painting (wdt:P136/wdt:P279*) wd:Q134307.
      ?painting wdt:P170 ?creator.
      ?painting p:P2048/psn:P2048 [wikibase:quantityAmount ?height;].
      ?painting p:P2049/psn:P2049 [wikibase:quantityAmount ?width;].
      ?painting wdt:P571 ?inception.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    GROUP BY ?painting ?paintingLabel
```

Note: the data files in this repository have a subset of the data only, with a few small modifications for demo purposes.

# Example data: portrait paintings

Queried from [Wikidata](https://www.wikidata.org), see notebook for details.

<img src='img/256px-Jean_Fouquet-_Portrait_of_the_Ferrara_Court_Jester_Gonella.JPG'/>

In [45]:
import pandas as pd
from tdda.constraints.pd.constraints import discover_df, verify_df, detect_df
from IPython.display import display

In [35]:
# Load the first set of example data

df = pd.read_csv('painting_data_01.csv')
df['inception'] = pd.to_numeric(df['inception'].str[:4])
display(df.head(3))
print(len(df))

,painting,paintingLabel,creator,width,height,inception
0,http://www.wikidata.org/entity/Q3937425,Portrait of the court jester Gonella,http://www.wikidata.org/entity/Q311398,0.24,0.36,1447
1,http://www.wikidata.org/entity/Q3937429,Pietro Secco Suardo,http://www.wikidata.org/entity/Q504949,1.04,1.83,1563
2,http://www.wikidata.org/entity/Q3937474,Portrait of Bianca Cappello,http://www.wikidata.org/entity/Q4233718,0.52,0.75,1560


31


In [63]:
# discover the constraints

constraints = discover_df(df).to_json()
print(constraints)
print('\n' * 15)  # reveal.js scroll fix...

{
    "creation_metadata": {
        "local_time": "2019-08-14 12:12:08",
        "utc_time": "2019-08-14 10:10:08",
        "creator": "TDDA 1.0.13",
        "host": "Eberhards-MacBook-Pro.local",
        "user": "ehansis",
        "n_records": 31,
        "n_selected": 31
    },
    "fields": {
        "painting": {
            "type": "string",
            "min_length": 39,
            "max_length": 39,
            "max_nulls": 0,
            "no_duplicates": true
        },
        "paintingLabel": {
            "type": "string",
            "min_length": 17,
            "max_length": 65,
            "max_nulls": 0
        },
        "creator": {
            "type": "string",
            "min_length": 35,
            "max_length": 39,
            "max_nulls": 0,
            "allowed_values": [
                "http://www.wikidata.org/entity/Q150679",
                "http://www.wikidata.org/entity/Q290407",
                "http://www.wikidata.org/entity/Q297",
                "htt

In [36]:
# save constraints to file

with open('constraints_01.json', 'w') as f:
    f.write(constraints)

In [38]:
# test constraints on our dataset

result = verify_df(df, 'constraints_01.json')
print(str(result))

FIELDS:

painting: 0 failures  5 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓  no_duplicates ✓

paintingLabel: 0 failures  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓

creator: 0 failures  5 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓  allowed_values ✓

width: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

height: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

inception: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

SUMMARY:

Constraints passing: 29
Constraints failing: 0


In [43]:
# Load a second set of test data

df = pd.read_csv('painting_data_02.csv')
df['inception'] = pd.to_numeric(df['inception'].str[:4])
display(df)
print(len(df))

,painting,paintingLabel,creator,width,height,inception
0,http://www.wikidata.org/entity/Q3937389,Portrait of a young man,http://www.wikidata.org/entity/Q8459,0.4600,0.5800,NaN
1,http://www.wikidata.org/entity/Q3794238,Q3794238,http://www.wikidata.org/entity/Q82445,0.5100,0.6100,1882.0
2,http://www.wikidata.org/entity/Q3937400,Portrait of a Man,http://www.wikidata.org/entity/Q312616,0.3112,0.4382,1475.0
3,http://www.wikidata.org/entity/Q3937399,Portrait of a Man,http://www.wikidata.org/entity/Q310973,0.9800,1.1500,1545.0
4,http://www.wikidata.org/entity/Q3822931,La madre profuga,http://www.wikidata.org/entity/Q3745137,0.9600,0.9100,1918.0


5


In [44]:
# test constraints

result = verify_df(df, 'constraints_01.json')
print(str(result))

FIELDS:

painting: 0 failures  5 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓  no_duplicates ✓

paintingLabel: 1 failure  3 passes  type ✓  min_length ✗  max_length ✓  max_nulls ✓

creator: 1 failure  4 passes  type ✓  min_length ✓  max_length ✓  max_nulls ✓  allowed_values ✗

width: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

height: 0 failures  5 passes  type ✓  min ✓  max ✓  sign ✓  max_nulls ✓

inception: 2 failures  3 passes  type ✓  min ✓  max ✗  sign ✓  max_nulls ✗

SUMMARY:

Constraints passing: 25
Constraints failing: 4


In [51]:
# use TDDA to show failures
detect_df(df, 'constraints_01.json', outpath='failures_02.csv', output_fields=[], per_constraint=True)
failures = pd.read_csv('failures_02.csv')
display(failures)

,painting,paintingLabel,creator,width,height,inception,paintingLabel_min_length_ok,creator_values_ok,inception_max_ok,inception_nonnull_ok,n_failures
0,http://www.wikidata.org/entity/Q3937389,Portrait of a young man,http://www.wikidata.org/entity/Q8459,0.4600,0.5800,NaN,True,True,NaN,False,1
1,http://www.wikidata.org/entity/Q3794238,Q3794238,http://www.wikidata.org/entity/Q82445,0.5100,0.6100,1882.0,False,False,False,True,3
2,http://www.wikidata.org/entity/Q3937400,Portrait of a Man,http://www.wikidata.org/entity/Q312616,0.3112,0.4382,1475.0,True,False,True,True,1
3,http://www.wikidata.org/entity/Q3822931,La madre profuga,http://www.wikidata.org/entity/Q3745137,0.9600,0.9100,1918.0,False,False,False,True,3


## Iterate on constraints and data

### Fix data:
* Correct one label
* Add one inception year (or remove non-null constraint)

### Fix constraints (edit JSON):
* Allow larger max for inception date
* Extend list of allowed creator values (or remove list of allowed values)

# Advanced constraints: RegEx discovery

* Fancy heuristics to discover regular expression patterns for fields
* Off by default

In [64]:
# build a constraint for the wikidata object URLs

df = pd.read_csv('painting_data_01.csv')
constraints = discover_df(df[['painting']], inc_rex=True).to_json()
print(constraints)
print('\n' * 15)

{
    "creation_metadata": {
        "local_time": "2019-08-14 12:12:16",
        "utc_time": "2019-08-14 10:10:16",
        "creator": "TDDA 1.0.13",
        "host": "Eberhards-MacBook-Pro.local",
        "user": "ehansis",
        "n_records": 31,
        "n_selected": 31
    },
    "fields": {
        "painting": {
            "type": "string",
            "min_length": 39,
            "max_length": 39,
            "max_nulls": 0,
            "no_duplicates": true,
            "rex": [
                "^http\\:\\/\\/www\\.wikidata\\.org\\/entity\\/[A-Z]\\d{7}$"
            ]
        }
    }
}



















## RegEx discovery...
* works really well in many cases
* is a good start for manual tweaking in others

## RegEx constraints can be very powerful!
* avoid quotation marks in input data
* force only numerical or non-numerical data
* enforce minimum number of words
* ...

# How I use this

* Luigi pipelines for ETL and other data processing
* (Nearly) every **processing task** has an associated **test task**, using TDDA plus other consistency checks
* Task requirements are always on the **test task** (downstream tasks only run if tests pass)

Example pipeline, <span style='color: #735BFB'>input</span>, <span style='color: #002982'>processing/wrapper</span> and <span style='color: #2ca79e'>test</span> tasks.

<img src='img/example_pipeline.png'/>

# Thank You

### Find this notebook at: https://github.com/ehansis/tdda-intro-2019

Eberhard Hansis, Vebeto GmbH, http://www.vebeto.de

eberhard.hansis@vebeto.de

<img src='https://www.vebeto.de/img/vebeto_green_blue_bm.svg'/>